In [1]:
#FULL CODE:
#Importing libraries
import sys
import os
import re
import time

import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import spacy

from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span

tqdm.pandas()
#using pretrained gensim model for creating vocabulary
import gensim.downloader as api
import gensim


In [2]:
import gensim
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import copy

In [3]:
#Loading different embeddings function by choice #files located locally
def load_embed(embed):
    if embed == "fasttext":
        return  (KeyedVectors.load('saved_ft_embed',mmap='r'))
    elif embed == "glove":
        return  (KeyedVectors.load('saved_glove_embed',mmap='r'))
    elif embed == "conceptnet":
        return (KeyedVectors.load(r"C:\Users\DELL\Desktop\NLP files\Ner model files\conceptnet_embeddings\saved_numb_batch_embed",mmap='r'))
    else:   
        return None

In [4]:
# {"nounchunks_info": [{
# "sentance-1":[
#     {"noun_chunk-1":{"root_word":"rape","embedding":"0.1"},
#     {"noun_chunk-2":{"root_word":"rape","embedding":"0.2"}
# ]},
# "sentance-2":[
#     {"noun_chunk-1":{"root_word":"rape","embedding":"0.1"},
#     {"noun_chunk-2":{"root_word":"rape","embedding":"0.2"}
# ]}]
# }

In [1]:
def LoadAndPreprocessInputsCorpus(corpus_filename):
    print("preprocess functions is runnning")
    # Readin the corpus file
    corpus_data = open(corpus_filename, encoding='utf-8').read().strip()

    # split into patents texts | 1 entry = 1 patent
    #corpus_texts = corpus_data.split('\n\n')

    # split each patent into lines
    corpus_lines = corpus_data.split('\n')
    print(type(corpus_lines))
    #print(len(corpus_lines))
    return corpus_lines

In [2]:
corpus_filename = r"C:\Users\DELL\Desktop\NLP files\Ner model files\sample_corpus_file.txt"
LoadAndPreprocessInputsCorpus(corpus_filename)

preprocess functions is runnning
<class 'list'>


['Shelf life extension?Woodside and its joint venture partners want to extend the life of the Karratha Gas Plant, which has been operating since 1984, so it can process third-party gas as the North West Shelf fields deplete.',
 'The Environmental Protection Authority has recommended ministerial approval but insisted on additional measures for the mitigation of greenhouse gas emissions to those proposed by the North West Shelf Joint Venture in its Greenhouse Gas Management Plan.']

In [6]:
def rootwordembed(corpus_filename):
    nlp = spacy.load("en_core_web_lg")
    embed_model = load_embed('conceptnet')
    corpus_lines = LoadAndPreprocessInputsCorpus(corpus_filename)
    nounchunks_info_dict={}
    sentence_dict_list=[]
    for line  in corpus_lines:
        doc = nlp(line)
        sentence_dict={}
        
        noun_chunks_lst=[]
        
        for chunk in doc.noun_chunks:
            if str(chunk.root.text) in embed_model.key_to_index:
                dict_1={}

                noun_chunk_rootembed_dict= {}
                noun_chunk_rootembed_dict['root_word']=str(chunk.root.text)
                noun_chunk_rootembed_dict['embedding']='embedding'#embed_model[str(chunk.root.text)]
                dict_1[str(chunk)]=noun_chunk_rootembed_dict
                noun_chunks_lst.append(copy.deepcopy(dict_1))#output  --- noun_chunks_lst
        sentence_dict[line]=noun_chunks_lst

        sentence_dict_list.append(copy.deepcopy(sentence_dict))

    nounchunks_info_dict['nounchunks_info']=sentence_dict_list
    return nounchunks_info_dict
    

In [7]:
corpus_filename = r"C:\Users\DELL\Desktop\NLP files\Ner model files\sample_corpus_file.txt"
samp=rootwordembed(corpus_filename)

preprocess functions is runnning


In [8]:
print(samp)

{'nounchunks_info': [{'Shelf life extension?Woodside and its joint venture partners want to extend the life of the Karratha Gas Plant, which has been operating since 1984, so it can process third-party gas as the North West Shelf fields deplete.': [{'Shelf life': {'root_word': 'life', 'embedding': 'embedding'}}, {'its joint venture partners': {'root_word': 'partners', 'embedding': 'embedding'}}, {'the life': {'root_word': 'life', 'embedding': 'embedding'}}, {'which': {'root_word': 'which', 'embedding': 'embedding'}}, {'it': {'root_word': 'it', 'embedding': 'embedding'}}, {'third-party gas': {'root_word': 'gas', 'embedding': 'embedding'}}, {'the North West Shelf fields': {'root_word': 'fields', 'embedding': 'embedding'}}]}, {'The Environmental Protection Authority has recommended ministerial approval but insisted on additional measures for the mitigation of greenhouse gas emissions to those proposed by the North West Shelf Joint Venture in its Greenhouse Gas Management Plan.': [{'minist

In [9]:
def LoadAndPreprocessInputsVocab(vocab_filename):
    print("preprocess functions is runnning")
    if vocab_filename.endswith('json'):
        f = open(vocab_filename)
        vocab_processed = json.load(f)
        
    return vocab_processed.

In [102]:
def keywordembed(vocab_filename):
    LABEL_NAME = 'GREEN_COMMITMENT'
    vocab_processed=LoadAndPreprocessInputsVocab(vocab_filename)
    expanded_keywords = vocab_processed[LABEL_NAME]
    embed_model = load_embed('conceptnet')
    keyword_embed_dict={}
    for keyword in list(expanded_keywords)[0:1272]:
        _keyword = keyword.replace(' ','_')
        if _keyword in embed_model.key_to_index:
            keyword_embed_dict[_keyword]=embed_model[_keyword]
    
    return keyword_embed_dict

In [103]:
vocab_filename =r"C:\Users\DELL\Desktop\NLP files\Ner model files\ESG_Keywords_Clean_4473.json"
green_commitment_embed=keywordembed(vocab_filename)

preprocess functions is runnning


In [104]:
len(green_commitment_embed.keys())

1023

In [105]:
 # cosine similarity function
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [141]:
#corpus_filename = r"C:\Users\DELL\Desktop\NLP files\Ner model files\sample_corpus_file.txt"
corpus_file = r"C:\Users\DELL\Desktop\NLP files\Ner model files\CE_GF_corpus_small_40.txt"
samp=word_embed(corpus_file)

  0%|                                                                                        | 0/12833 [00:00<?, ?it/s]

preprocess functions is runnning


100%|███████████████████████████████████████████████████████████████████████████| 12833/12833 [01:03<00:00, 202.42it/s]


In [142]:
print(len(samp))

13273


In [139]:
#print(samp)
from tqdm import tqdm

In [140]:
def word_embed(corpus_filename):
    nlp = spacy.load("en_core_web_lg")
    embed_model = load_embed('conceptnet')
    corpus_lines = LoadAndPreprocessInputsCorpus(corpus_filename)
    noun_chunk_dict={}
    for line  in tqdm(corpus_lines):
        doc = nlp(line)
        
        for chunk in doc.noun_chunks:
            
            if str(chunk.root.text) in embed_model.key_to_index:
                noun_chunk_dict[str(chunk)]=embed_model[str(chunk.root.text)]
    return noun_chunk_dict
                
                

In [148]:
def chunk_comparision(noun_chunk_dict,keyword_embed_dict):
    counter=0
    similarity_dict={}
    similarity_dict['noun_chunk']=[]
    #similarity_dict['similarity_score']=[]
    
    for noun_chunk,noun_chunk_embed  in tqdm(noun_chunk_dict.items()):
        for key_word,key_embedding in keyword_embed_dict.items():
            similarity = get_cosine_similarity(key_embedding, noun_chunk_embed)
                        
            if similarity > 0.6:
                similarity_dict['noun_chunk'].append(noun_chunk)
                #similarity_dict['similarity_score'].append(similarity)
                counter+=1
    return similarity_dict       

In [149]:
%%time
output_dict=chunk_comparision(samp,green_commitment_embed)

100%|████████████████████████████████████████████████████████████████████████████| 13273/13273 [31:31<00:00,  7.02it/s]

Wall time: 31min 31s


In [151]:
print(len(output_dict['noun_chunk']))

7486


In [153]:
print(len(set(output_dict['noun_chunk'])))

2744


In [156]:
len(set(output_dict['noun_chunk'][0:20]))

4

In [ ]:
import json
path=r"C:\Users\DELL\Desktop\ESG_Keywords_Clean_4473.json"
with open(path, 'w') as f:
    json.dump(new_dict, f)

In [113]:
print(word_embed(corpus_filename))

preprocess functions is runnning
{'Shelf life': memmap([-0.09310227, -0.04270104, -0.05290129, -0.05130126, -0.07620186,
         0.07370181,  0.02240055,  0.05240128,  0.20310496,  0.10840265,
        -0.08140199,  0.02750067,  0.01130028, -0.0616015 , -0.11820289,
         0.01340033,  0.11680286,  0.1025025 , -0.06620162, -0.08260202,
         0.11200274,  0.01050026,  0.01960048, -0.05380132, -0.0776019 ,
        -0.11820289, -0.10990268,  0.02770068, -0.09320228,  0.07430182,
        -0.08340205,  0.05200127,  0.05400132,  0.08870216,  0.0285007 ,
        -0.05610137,  0.00900022, -0.00160004, -0.01140028,  0.01560038,
         0.10570259,  0.13230324,  0.03840094,  0.02790068, -0.01470036,
         0.0082002 , -0.03510086, -0.15270373,  0.05200127,  0.00690017,
        -0.02890071,  0.05990146,  0.02750067,  0.04020098,  0.0285007 ,
         0.09250226, -0.06620162, -0.07950194,  0.04800117, -0.03430084,
         0.05600137,  0.01370033,  0.02790068,  0.15230373, -0.05840143,
   

In [83]:
def word_embed(corpus_filename):
    nlp = spacy.load("en_core_web_lg")
    embed_model = load_embed('conceptnet')
    corpus_lines = LoadAndPreprocessInputsCorpus(corpus_filename)
    lst=[]    
    for line  in corpus_lines:
        doc = nlp(line)
        
        for chunk in doc.noun_chunks:
            dict_1={}
            if str(chunk.root.text) in embed_model.key_to_index:
                
                noun_chunk_rootembed_dict= {}
                noun_chunk_rootembed_dict[str(chunk.root.text)]=embed_model[str(chunk.root.text)]
                #noun_chunk_rootembed_dict['root_word']=str(chunk.root.text)
                #noun_chunk_rootembed_dict['embedding']=embed_model[str(chunk.root.text)]
                dict_1[str(chunk)]=noun_chunk_rootembed_dict
            
                lst.append(copy.deepcopy(dict_1))
    return lst


In [84]:
corpus_file = r"C:\Users\DELL\Desktop\NLP files\Ner model files\CE_GF_corpus_small_40.txt"
rootword_embed_list=word_embed(corpus_file)
#print(rootword_embed_list)
len(rootword_embed_list)

preprocess functions is runnning


26724

In [97]:
with open(r"C:\Users\DELL\Desktop\NLP files\Ner model files\CE_GF_corpus_small_40.txt",'r',encoding='utf8') as f:
    text=f.read()
words = text.split()
print(len(words))

137386


In [114]:
#rootword_embed_list[26000]

In [115]:
# for dictionary in rootword_embed_list:
#     for noun_chunk in dictionary.keys():
#         print(noun_chunk)
#         for value in dictionary.values():
#             for key,value in value.items():
#                 print(key,value)

In [80]:
def chunk_comparision(nounchunks_info_list,keyword_embed_dict):
    counter=0
    similarity_dict={}
    similarity_dict['noun_chunk']=[]
    similarity_dict['similarity_score']=[]
    
    for dictionary in nounchunks_info_list:
        for noun_chunk in dictionary.keys():
            
            for value in dictionary.values():
                for root_key,root_value in value.items():
                    #print("root_value--1",root_value)
                    for key_word,key_embedding in keyword_embed_dict.items():
#                         print(type(key_embedding))
#                         print("root_value--2:", root_value)
                        
                        similarity = get_cosine_similarity(key_embedding, root_value)
                        
                        if similarity > 0.6:
                            similarity_dict['noun_chunk'].append(noun_chunk)
                            similarity_dict['similarity_score'].append(similarity)
                            counter+=1
    return similarity_dict

In [106]:
%%time
out_dict=chunk_comparision(rootword_embed_list,green_commitment_embed)

KeyboardInterrupt: 

In [82]:
#print(out_dict)

{'noun_chunk': ['greenhouse gas emissions', 'greenhouse gas emissions', 'greenhouse gas emissions', 'greenhouse gas emissions', 'greenhouse gas emissions', 'reservoir carbon dioxide emissions', 'reservoir carbon dioxide emissions', 'reservoir carbon dioxide emissions', 'reservoir carbon dioxide emissions', 'reservoir carbon dioxide emissions', 'net scope 1 greenhouse gas emissions', 'net scope 1 greenhouse gas emissions', 'net scope 1 greenhouse gas emissions', 'net scope 1 greenhouse gas emissions', 'net scope 1 greenhouse gas emissions', '1 and 2 greenhouse gas emissions', '1 and 2 greenhouse gas emissions', '1 and 2 greenhouse gas emissions', '1 and 2 greenhouse gas emissions', '1 and 2 greenhouse gas emissions', 'energy', 'energy', 'their decarbonisation commitments', 'their decarbonisation commitments', 'their decarbonisation commitments', 'their decarbonisation commitments', 'their decarbonisation commitments', 'their decarbonisation commitments', 'their decarbonisation commitmen

In [98]:
print(len(out_dict['noun_chunk']))

1006


In [100]:
a=set(out_dict['noun_chunk'])
len(a)

112

In [101]:
print(a)

{'greenhouse gas pollution', 'environmental pollution', '-- Low carbon sustainable energy', 'renewables', 'both external and internal indicators', 'the European sustainable energy', 'net-zero greenhouse gas (GHG) emissions', 'commitment', 'climate', 'Equita commitment', 'its emissions', '1 and 2 greenhouse gas emissions', 'other business commitments', 'American-made energy', '    commitments', 'harmful pollution', 'carbon dioxide emissions', 'the two oaths', 'energy', 'commitments', 'Scope 3 portfolio emissions', 'its decarbonisation commitments', "its portfolio's GHG emissions", '82 per cent renewables', 'the source', 'more cheap and reliable firmed renewables', 'greenhouse gases', 'the promise', 'the indicators', 'capital commitments', 'specific impairment indicators', 'their emissions', 'net scope 1 greenhouse gas emissions', 'a KPI indicators', 'whether any indication', 'reservoir carbon dioxide emissions', 'CO2 emissions', 'US greenhouse gas emissions', 'a renewables', "the Group'

In [ ]:
df=pd.read_excel()

In [20]:
corpus=['This is line1','This is line2','This is line3','This is line4']



#nounchunks_info_dict['nounchunks_info']=nounchunks_info_list
nounchunks_info_dict={}
sentence_dict_list=[]
for sent in corpus:
        
        sentence_dict={}
        
        noun_chunks=["chunk1","chunk2","chunk3"]
        noun_chunks_lst=[]
        for noun_chunk in noun_chunks:
            dict_1={}
            
            noun_chunk_rootembed_dict= {}
            noun_chunk_rootembed_dict['root_word']='ROOT WORD'
            noun_chunk_rootembed_dict['embedding']='EMBED'
            dict_1[noun_chunk]=noun_chunk_rootembed_dict
            noun_chunks_lst.append(copy.deepcopy(dict_1))#output  --- noun_chunks_lst
        sentence_dict[sent]=noun_chunks_lst
        
        sentence_dict_list.append(copy.deepcopy(sentence_dict))

nounchunks_info_dict['nounchunks_info']=sentence_dict_list
print(nounchunks_info_dict)

{'nounchunks_info': [{'This is line1': [{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk2': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk3': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}]}, {'This is line2': [{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk2': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk3': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}]}, {'This is line3': [{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk2': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk3': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}]}, {'This is line4': [{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk2': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk3': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}]}]}


In [21]:
def getRootWordEmbedding(nounchunks_info_dict):
    for sentence_dict in next(iter(nounchunks_info_dict.items()))[1]:
        for chunk_dict in next(iter(sentence_dict.items()))[1]:
            print(chunk_dict)
            for chunk in chunk_dict.values():
                return (chunk['root_word'],chunk['embedding'])


In [22]:
getRootWordEmbedding(nounchunks_info_dict)

{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}


('ROOT WORD', 'EMBED')

In [ ]:
{'nounchunks_info': [{'This is line1': [{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk2': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk3': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}]}, {'This is line2': [{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk2': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk3': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}]}, {'This is line3': [{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk2': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk3': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}]}, {'This is line4': [{'chunk1': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk2': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}, {'chunk3': {'root_word': 'ROOT WORD', 'embedding': 'EMBED'}}]}]}

In [5]:
print("abc")

abc
